In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Flatten, MaxPooling2D, Conv2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import matplotlib.pyplot as plt
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
datagen = ImageDataGenerator(rescale=1.0 / 255.0)
train_generator = datagen.flow_from_directory(
    '/mnt/d/My projects/programs/Python projects/dbi-ai-model/res/archive_300x300_30class/dogImages/dogImages/train',
    #'res\\archive_300x300_30class\dogImages\dogImages\\train',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical')

valid_generator = datagen.flow_from_directory(
    '/mnt/d/My projects/programs/Python projects/dbi-ai-model/res/archive_300x300_30class/dogImages/dogImages/valid',
    #'res\\archive_300x300_30class\dogImages\dogImages\\valid',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical')

In [ ]:
model = keras.Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same", activation='relu', input_shape=(300, 300, 3)),#300
        MaxPooling2D((3, 3)),
        Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same", activation='relu'),#100
        MaxPooling2D((3, 3)),
        Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same", activation='relu'),#33
        MaxPooling2D((3, 3)),
        Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same", activation='relu'),#11
        MaxPooling2D((3, 3)),
        Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same", activation='relu'),#3
        MaxPooling2D((3, 3)),
        Flatten(),
        Dense(units=50, activation="relu"),
        Dropout(0.5),
        Dense(units=50, activation="relu"),
        Dropout(0.5),
        Dense(units=30, activation="softmax")
    ])

model.summary()

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

In [ ]:
checkpoint = ModelCheckpoint('best_weights.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min', verbose=1)

In [ ]:
model.load_weights("best_weights.h5")

In [ ]:
model_result = model.fit(train_generator,
                         epochs=20,
                         validation_data=valid_generator,
                         #validation_data=(valid_input, valid_output)
                         callbacks=[checkpoint]
                         )

In [ ]:
model.evaluate(train_generator)

In [ ]:
model.evaluate(valid_generator)

In [ ]:
test_generator = datagen.flow_from_directory(
    #'res\\archive_300x300_30class\dogImages\dogImages\\test',
    '/mnt/d/My projects/programs/Python projects/dbi-ai-model/res/archive_300x300_30class/dogImages/dogImages/test',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical')

model.evaluate(test_generator)

In [ ]:
plt.plot(model_result.history['loss'], label='Тренувальні втрати')
plt.plot(model_result.history['val_loss'], label='Валідаційні втрати')
plt.xlabel('Епоха')
plt.ylabel('Втрати')
plt.legend()
plt.show()

plt.plot(model_result.history['accuracy'], label='Тренувальна точність')
plt.plot(model_result.history['val_accuracy'], label='Валідаційна точність')
plt.xlabel('Епоха')
plt.ylabel('Точність')
plt.legend()
plt.show()